In [ ]:
# WHAM Library
import numpy as np
import scipy
import itertools
import  pathlib 

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.interactive(True)
#%config InlineBackend.figure_format = 'pdf'
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from matplotlib import gridspec

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 300

plt.rcParams['figure.autolayout'] = False

plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams['font.serif'] = "cm"
plt.rcParams['text.latex.preamble'] = [
    r"\usepackage{subdepth}",
    r"\usepackage{type1cm}",
    r'\usepackage{tgheros}',    # helvetica font
    r'\usepackage{sansmath}',   # math-font matching  helvetica
    r'\sansmath'                # actually tell tex to use it!
#    r'\usepackage{siunitx}',    # micro symbols
#    r'\sisetup{detect-all}',    # force siunitx to use the fonts
]  

print(plt.rcParams['figure.figsize'])


def set_size(column=0):
    if (column==0):
        plt.rcParams['figure.figsize']   = [10,6]
        plt.rcParams['axes.labelsize']   = 18 
        plt.rcParams['axes.titlesize']   = 20 
        plt.rcParams['font.size']        = 16 
        plt.rcParams['lines.linewidth']  = 2.0 
        plt.rcParams['lines.markersize'] = 8 
        plt.rcParams['legend.fontsize']  = 14 
    elif (column==1):
        plt.rcParams['figure.figsize']   = [3.5,3.5*0.6]
        plt.rcParams['axes.labelsize']   = 10 
        plt.rcParams['axes.titlesize']   = 12 
        plt.rcParams['font.size']        = 10 
        plt.rcParams['lines.linewidth']  = 1.5 
        plt.rcParams['lines.markersize'] = 10 
        plt.rcParams['legend.fontsize']  = 10 
    elif (column==2):
        plt.rcParams['figure.figsize']   = [7.0,7.0*0.6]
        plt.rcParams['axes.labelsize']   = 10 
        plt.rcParams['axes.titlesize']   = 12 
        plt.rcParams['font.size']        = 10 
        plt.rcParams['lines.linewidth']  = 2.0 
        plt.rcParams['lines.markersize'] = 10 
        plt.rcParams['legend.fontsize']  = 10 
        
set_size(column=2)

In [ ]:
dir='../data/d_tor1_tor2/'
N_windows = 1370

w_i = np.array([], dtype="int")
t   = np.array([])
q = {}
q["d"], q["tor1"], q["tor2"] = np.array([]), np.array([]), np.array([])

for window in range(0, N_windows):
    print(window, end=" ")
    try:
        c1, c2, c3, c4 = np.loadtxt(dir + str(window) + ".txt", unpack=True)
        w_i  = np.append(w_i, np.full(c1.shape, window))
        t    = np.append(t, c1)
        q["d"]    = np.append(q["d"]   , c2)
        q["tor1"] = np.append(q["tor1"], c3)
        q["tor2"] = np.append(q["tor2"], c4)
    except:
        continue
    


In [ ]:
# plot them 
plt.figure()
plt.plot(q["d"])
plt.xlabel(r"sample index")
plt.ylabel(r"r / nm")

plt.figure()
plt.plot(q["tor1"])
plt.xlabel(r"sample index")
plt.ylabel(r"$\mathrm{\Theta}^0$ / $^\circ$")

plt.figure()
plt.plot(q["tor2"])
plt.xlabel(r"sample index")
plt.ylabel(r"$\mathrm{\Theta}^1$ / $^\circ$")


In [ ]:
dir='../data/plumed_params/'
window_list=np.unique(w_i)

p={}
p["d"], p["tor1"], p["tor2"] = np.array([]), np.array([]), np.array([])

for window in window_list:
    print(window, end=" ")
    c1, c2, c3 = np.loadtxt(dir + str(window) + ".txt", unpack=True).T
    p["d"]    = np.append(p["d"],    c1)
    p["tor1"] = np.append(p["tor1"], c2)
    p["tor2"] = np.append( p["tor2"], c3)

p["d"]    = p["d"]   .reshape(-1,2)
p["tor1"] = p["tor1"].reshape(-1,2)
p["tor2"] = p["tor2"].reshape(-1,2)

In [ ]:
U = {}

In [ ]:
q_t = q["tor1"].reshape(-1, 1)
q_0 = p["tor1"][:,0].reshape(1, -1)
k   = p["tor1"][:,1].reshape(1, -1)

X = q_t - q_0
# taking the periodicity into account
X[X> np.pi] -= 2*np.pi
X[X<-np.pi] += 2*np.pi

U["tor1"] = 0.5 * k * X**2.0

In [ ]:
q_t = q["tor2"].reshape(-1, 1)
q_0 = p["tor2"][:,0].reshape(1, -1)
k   = p["tor2"][:,1].reshape(1, -1)

X = q_t - q_0
# taking the periodicity into account
X[X> np.pi] -= 2*np.pi
X[X<-np.pi] += 2*np.pi

U["tor2"] = 0.5 * k * X**2.0

In [ ]:
q_t = q["d"].reshape(-1, 1)
q_0 = p["d"][:,0].reshape(1, -1)
k   = p["d"][:,1].reshape(1, -1)

X = q_t - q_0
# settign all negative distances to 0 for half harmonic
X[X<0] = 0

U["d"] = k * X**2.0

In [ ]:
U["total"] = U["tor1"] + U["tor2"] + U["d"]

In [ ]:
mat = U["d"][::10, ::1]

plt.figure()
square=mat.shape[1]/mat.shape[0]
plt.imshow(mat, aspect=square, origin='upper',interpolation='none')

plt.gca().xaxis.set_ticks_position('bottom')
plt.ylabel(r"sample index")
plt.xlabel(r"window index")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"U / (kJ/mol)")
plt.title(r'Bias Matrix', y=1.05)

In [ ]:
mat = U["tor2"][::10, ::1]

plt.figure()
square=mat.shape[1]/mat.shape[0]
plt.imshow(mat, aspect=square, origin='upper',interpolation='none')

plt.gca().xaxis.set_ticks_position('bottom')
plt.ylabel(r"sample index")
plt.xlabel(r"window index")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"U / (kJ/mol)")
plt.title(r'Bias Matrix', y=1.05)

In [ ]:
mat = U["tor1"][::10, ::1]

plt.figure()
square=mat.shape[1]/mat.shape[0]
plt.imshow(mat, aspect=square, origin='upper',interpolation='none')

plt.gca().xaxis.set_ticks_position('bottom')
plt.ylabel(r"sample index")
plt.xlabel(r"window index")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"U / (kJ/mol)")
plt.title(r'Bias Matrix', y=1.05)


In [ ]:
mat = U["total"][::10, ::1]

plt.figure()
square=mat.shape[1]/mat.shape[0]
plt.imshow(mat, aspect=square, origin='upper',interpolation='none')

plt.gca().xaxis.set_ticks_position('bottom')
plt.ylabel(r"sample index")
plt.xlabel(r"window index")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"U / (kJ/mol)")
plt.title(r'Bias Matrix', y=1.05)

In [ ]:
np.save("U.npy", U["total"])

In [ ]:
np.save("w_i.npy", w_i)